In [3]:
"""
9:30-10:45 AM RANGE BACKTEST - TODAY ONLY
Uses 15-min candles from 9:30-10:45 AM to establish range
20 Contracts | 5pt SL | 40pt TP (8:1 R/R)
"""
import os
import requests
from dotenv import load_dotenv
from datetime import datetime, timedelta
import pytz

load_dotenv()

USERNAME = os.getenv("TOPSTEP_USERNAME")
KEY = os.getenv("TOPSTEP_KEY")
API_BASE = "https://api.topstepx.com/api"
CONTRACT_ID = "CON.F.US.MNQ.H26"

# Strategy parameters
POSITION_SIZE = 20
FIXED_STOP_LOSS = 5
FIXED_TAKE_PROFIT = 40

# ==================== API FUNCTIONS ====================
def authenticate():
    resp = requests.post(
        f"{API_BASE}/Auth/loginKey",
        json={"userName": USERNAME, "apiKey": KEY},
        headers={"accept": "text/plain", "Content-Type": "application/json"},
        timeout=15
    )
    return {
        "Authorization": f"Bearer {resp.json()['token']}",
        "accept": "text/plain",
        "Content-Type": "application/json"
    }

def fetch_bars(headers, unit, unit_number):
    end_time = datetime.now(pytz.UTC)
    start_time = end_time - timedelta(days=2)
    
    payload = {
        "contractId": CONTRACT_ID,
        "live": False,
        "startTime": start_time.strftime("%Y-%m-%dT%H:%M:%SZ"),
        "endTime": end_time.strftime("%Y-%m-%dT%H:%M:%SZ"),
        "unit": unit,
        "unitNumber": unit_number,
        "limit": 10000,
        "includePartialBar": True
    }
    
    resp = requests.post(f"{API_BASE}/History/retrieveBars", json=payload, headers=headers, timeout=30)
    return resp.json().get("bars", [])

def parse_bars_with_ny_time(bars):
    ny_tz = pytz.timezone('America/New_York')
    parsed = []
    
    for bar in bars:
        timestamp = bar['t'].replace('+00:00', 'Z')
        dt_utc = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%SZ")
        dt_utc = pytz.utc.localize(dt_utc)
        dt_ny = dt_utc.astimezone(ny_tz)
        
        bar['dt_ny'] = dt_ny
        bar['date'] = dt_ny.date()
        parsed.append(bar)
    
    return parsed

def get_morning_range(bars_15m, today):
    """Get the range from 9:30 AM - 10:45 AM using 15-minute candles"""
    print("\n🔍 DEBUG: All 15-min bars for today around 9:30-10:45 AM:")
    
    # Find all 15-min bars between 9:30 AM and 10:45 AM (inclusive)
    morning_bars = []
    for bar in bars_15m:
        if bar['date'] == today:
            hour = bar['dt_ny'].hour
            minute = bar['dt_ny'].minute
            
            # 9:30 AM = hour 9, minute 30
            # 10:45 AM = hour 10, minute 45
            # We want bars that START at 9:30, 9:45, 10:00, 10:15, 10:30
            if (hour == 9 and minute >= 30) or (hour == 10 and minute <= 30):
                morning_bars.append(bar)
                print(f"   {bar['dt_ny'].strftime('%I:%M %p')} - H: {bar['h']:.2f}, L: {bar['l']:.2f}")
    
    if not morning_bars:
        return None
    
    # Calculate the range high and low
    range_high = max(bar['h'] for bar in morning_bars)
    range_low = min(bar['l'] for bar in morning_bars)
    
    print(f"\n✅ Morning Range (9:30 AM - 10:45 AM):")
    print(f"   High: {range_high:.2f}")
    print(f"   Low: {range_low:.2f}")
    print(f"   Size: {range_high - range_low:.2f} points")
    
    return {
        'high': range_high,
        'low': range_low
    }

def simulate_trade(direction, entry, stop, target, future_bars):
    for bar in future_bars:
        if direction == 'long':
            if bar['l'] <= stop:
                pnl = (stop - entry) * 0.5 * POSITION_SIZE
                return {'result': 'LOSS', 'pnl': pnl, 'exit_price': stop}
            if bar['h'] >= target:
                pnl = (target - entry) * 0.5 * POSITION_SIZE
                return {'result': 'WIN', 'pnl': pnl, 'exit_price': target}
        else:
            if bar['h'] >= stop:
                pnl = (entry - stop) * 0.5 * POSITION_SIZE
                return {'result': 'LOSS', 'pnl': pnl, 'exit_price': stop}
            if bar['l'] <= target:
                pnl = (entry - target) * 0.5 * POSITION_SIZE
                return {'result': 'WIN', 'pnl': pnl, 'exit_price': target}
    
    # Still open
    exit_price = future_bars[-1]['c'] if future_bars else entry
    if direction == 'long':
        pnl = (exit_price - entry) * 0.5 * POSITION_SIZE
    else:
        pnl = (entry - exit_price) * 0.5 * POSITION_SIZE
    
    return {'result': 'OPEN', 'pnl': pnl, 'exit_price': exit_price}

# ==================== BACKTEST TODAY ====================
def backtest_today(bars_15m, bars_5m, today):
    trades = []
    
    # Get morning range (9:30 AM - 10:30 AM)
    range_data = get_morning_range(bars_15m, today)
    
    if range_data is None:
        return None, "No morning range found for today"
    
    range_high = range_data['high']
    range_low = range_data['low']
    
    # Get today's 5-minute bars AFTER 10:45 AM (when the range completes)
    bars_today = []
    for b in bars_5m:
        if b['date'] == today:
            hour = b['dt_ny'].hour
            minute = b['dt_ny'].minute
            # After 10:45 AM means hour > 10, or hour == 10 and minute >= 45
            if hour > 10 or (hour == 10 and minute >= 45):
                bars_today.append(b)
    
    # IMPORTANT: Reverse to chronological order (oldest first)
    bars_today = list(reversed(bars_today))
    
    if len(bars_today) < 10:
        return None, "Not enough 5m bars after 10:45 AM"
    
    # Scan for setups
    print(f"\nScanning {len(bars_today)} bars for setups (after 10:45 AM)...")
    print(f"Range: {range_high:.2f} (high) - {range_low:.2f} (low)")
    print(f"Range Size: {range_high - range_low:.2f} points\n")
    print("Showing first 20 bars after 10:45 AM...")
    print("-" * 80)
    
    for i in range(min(20, len(bars_today))):
        bar = bars_today[i]
        status = ""
        if bar['c'] > range_high:
            status = "ABOVE range"
        elif bar['c'] < range_low:
            status = "BELOW range"
        else:
            status = "INSIDE range"
        
        print(f"{bar['dt_ny'].strftime('%I:%M %p')} - Close: {bar['c']:.2f} ({status})")
    
    print("-" * 80)
    print("\nNow checking for setups...\n")
    
    for i in range(len(bars_today) - 1):
        current_bar = bars_today[i]
        next_bar = bars_today[i + 1]
        
        # Step 1: Check if current bar closed OUTSIDE range
        closed_above = current_bar['c'] > range_high
        closed_below = current_bar['c'] < range_low
        
        # Step 2: Check if NEXT bar closed back INSIDE range
        next_closed_inside = range_low <= next_bar['c'] <= range_high
        
        # Debug: Print when we find potential setups
        if closed_above and next_closed_inside:
            print(f"[FOUND SHORT SETUP]")
            print(f"  Breakout Bar: {current_bar['dt_ny'].strftime('%I:%M %p')} - Close: {current_bar['c']:.2f} (ABOVE {range_high:.2f})")
            print(f"  Entry Bar: {next_bar['dt_ny'].strftime('%I:%M %p')} - Close: {next_bar['c']:.2f} (back INSIDE)")
            print()
        
        if closed_below and next_closed_inside:
            print(f"[FOUND LONG SETUP]")
            print(f"  Breakout Bar: {current_bar['dt_ny'].strftime('%I:%M %p')} - Close: {current_bar['c']:.2f} (BELOW {range_low:.2f})")
            print(f"  Entry Bar: {next_bar['dt_ny'].strftime('%I:%M %p')} - Close: {next_bar['c']:.2f} (back INSIDE)")
            print()
        
        # SHORT SETUP: Broke above, came back in
        if closed_above and next_closed_inside:
            entry_price = next_bar['c']
            stop_loss = entry_price + FIXED_STOP_LOSS
            take_profit = entry_price - FIXED_TAKE_PROFIT
            
            future_bars = bars_today[i+2:]
            outcome = simulate_trade('short', entry_price, stop_loss, take_profit, future_bars)
            
            trades.append({
                'time': next_bar['dt_ny'],
                'type': 'SHORT',
                'entry': entry_price,
                'stop': stop_loss,
                'target': take_profit,
                'result': outcome['result'],
                'pnl': outcome['pnl'],
                'exit_price': outcome['exit_price'],
                'breakout_bar_time': current_bar['dt_ny'],
                'entry_bar_time': next_bar['dt_ny']
            })
        
        # LONG SETUP: Broke below, came back in
        elif closed_below and next_closed_inside:
            entry_price = next_bar['c']
            stop_loss = entry_price - FIXED_STOP_LOSS
            take_profit = entry_price + FIXED_TAKE_PROFIT
            
            future_bars = bars_today[i+2:]
            outcome = simulate_trade('long', entry_price, stop_loss, take_profit, future_bars)
            
            trades.append({
                'time': next_bar['dt_ny'],
                'type': 'LONG',
                'entry': entry_price,
                'stop': stop_loss,
                'target': take_profit,
                'result': outcome['result'],
                'pnl': outcome['pnl'],
                'exit_price': outcome['exit_price'],
                'breakout_bar_time': current_bar['dt_ny'],
                'entry_bar_time': next_bar['dt_ny']
            })
    
    return trades, (range_high, range_low)

# ==================== MAIN ====================
def main():
    print("=" * 60)
    print("TODAY'S BACKTEST - 9:30-10:45 AM RANGE STRATEGY")
    print("20 Contracts | 5pt SL | 40pt TP")
    print("=" * 60)
    
    # Get today's date
    ny_tz = pytz.timezone('America/New_York')
    today = datetime.now(ny_tz).date()
    
    print(f"\n📅 Testing: {today}")
    
    print("\n[1/3] Authenticating...")
    headers = authenticate()
    print("✅ Authenticated")
    
    print("\n[2/3] Fetching data...")
    bars_15m = fetch_bars(headers, unit=2, unit_number=15)
    bars_5m = fetch_bars(headers, unit=2, unit_number=5)
    print(f"✅ Fetched {len(bars_15m)} 15m bars, {len(bars_5m)} 5m bars")
    
    bars_15m = parse_bars_with_ny_time(bars_15m)
    bars_5m = parse_bars_with_ny_time(bars_5m)
    
    print("\n[3/3] Running backtest...")
    trades, range_or_error = backtest_today(bars_15m, bars_5m, today)
    
    if trades is None:
        print(f"❌ {range_or_error}")
        return
    
    range_high, range_low = range_or_error
    
    print("✅ Backtest complete")
    
    print("\n" + "=" * 60)
    print("RESULTS")
    print("=" * 60)
    
    print(f"\n📊 9:30-10:45 AM RANGE:")
    print(f"   HIGH: {range_high:.2f}")
    print(f"   LOW:  {range_low:.2f}")
    print(f"   SIZE: {range_high - range_low:.2f} points")
    
    if not trades:
        print("\n⚠️ No valid setups found today")
        print("   (No breakout + re-entry occurred)")
        return
    
    # Calculate stats
    wins = [t for t in trades if t['result'] == 'WIN']
    losses = [t for t in trades if t['result'] == 'LOSS']
    open_trades = [t for t in trades if t['result'] == 'OPEN']
    
    total_pnl = sum(t['pnl'] for t in trades)
    
    print(f"\n📈 TRADES: {len(trades)}")
    print(f"   Wins: {len(wins)}")
    print(f"   Losses: {len(losses)}")
    print(f"   Still Open: {len(open_trades)}")
    
    print(f"\n💰 TOTAL P&L: ${total_pnl:.2f}")
    
    print("\n" + "=" * 60)
    print("ALL TRADES")
    print("=" * 60)
    
    for trade in trades:
        if trade['result'] == 'WIN':
            symbol = "✅"
        elif trade['result'] == 'LOSS':
            symbol = "❌"
        else:
            symbol = "⏳"
        
        print(f"\n{symbol} {trade['type']} @ {trade['time'].strftime('%I:%M %p')}")
        print(f"   Breakout: {trade['breakout_bar_time'].strftime('%I:%M %p')} (closed outside)")
        print(f"   Entry: {trade['entry_bar_time'].strftime('%I:%M %p')} (closed back in)")
        print(f"   Entry Price: {trade['entry']:.2f} | Exit: {trade['exit_price']:.2f}")
        print(f"   Stop: {trade['stop']:.2f} | Target: {trade['target']:.2f}")
        print(f"   Result: {trade['result']}")
        print(f"   P&L: ${trade['pnl']:.2f}")
    
    print("\n" + "=" * 60)
    print("✅ DONE")
    print("=" * 60)

if __name__ == "__main__":
    main()

TODAY'S BACKTEST - 9:30-10:45 AM RANGE STRATEGY
20 Contracts | 5pt SL | 40pt TP

📅 Testing: 2026-01-29

[1/3] Authenticating...
✅ Authenticated

[2/3] Fetching data...
✅ Fetched 185 15m bars, 553 5m bars

[3/3] Running backtest...

🔍 DEBUG: All 15-min bars for today around 9:30-10:45 AM:
   10:30 AM - H: 25753.50, L: 25609.00
   10:15 AM - H: 25822.00, L: 25720.00
   10:00 AM - H: 25868.50, L: 25718.25
   09:45 AM - H: 26008.00, L: 25858.25
   09:30 AM - H: 26177.25, L: 25946.00

✅ Morning Range (9:30 AM - 10:45 AM):
   High: 26177.25
   Low: 25609.00
   Size: 568.25 points

Scanning 43 bars for setups (after 10:45 AM)...
Range: 26177.25 (high) - 25609.00 (low)
Range Size: 568.25 points

Showing first 20 bars after 10:45 AM...
--------------------------------------------------------------------------------
10:45 AM - Close: 25602.50 (BELOW range)
10:50 AM - Close: 25545.50 (BELOW range)
10:55 AM - Close: 25576.75 (BELOW range)
11:00 AM - Close: 25559.50 (BELOW range)
11:05 AM - Close: 